<a href="https://colab.research.google.com/github/Annie-Yeeun-Jang/Info_retrieval_project/blob/master/%5BSI650_project%5D_MF_leave_one_out.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse.linalg import svds


In [5]:
jester=pd.read_excel('./FINAL jester 2006-15.xls', header=None) #ratings matrix with nan and range -10~+10 

In [6]:
jester

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,62.0,NaN,NaN,NaN,NaN,0.21875,NaN,-9.28125,-9.28125,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34.0,NaN,NaN,NaN,NaN,-9.68750,NaN,9.93750,9.53125,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18.0,NaN,NaN,NaN,NaN,-9.84375,NaN,-9.84375,-7.21875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,82.0,NaN,NaN,NaN,NaN,6.90625,NaN,4.75000,-5.90625,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27.0,NaN,NaN,NaN,NaN,-0.03125,NaN,-9.09375,-0.40625,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54900,13.0,NaN,NaN,NaN,NaN,NaN,NaN,-6.53125,-2.34375,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54901,8.0,NaN,NaN,NaN,NaN,NaN,NaN,8.93750,9.78125,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54902,27.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.59375,4.53125,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54903,8.0,NaN,NaN,NaN,NaN,NaN,NaN,-7.40625,6.93750,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
jester.notnull().sum()[:20] #they are some jokes that nobody rated [1,2,3,4,6,9,10,11,12,14]

0     54820
1         0
2         0
3         0
4         0
5       566
6         0
7     54905
8     54905
9         0
10        0
11        0
12        0
13    54905
14        0
15    54905
16    54905
17    54905
18    54905
19    54905
dtype: int64

### we need to do some preprocessing (users only having negative ratings or few ratings(<5), jokes that have never been rated)

In [10]:
#dropping jokes that have never been rated
proc=jester.drop(columns=[1,2,3,4,6,9,10,11,12,14], axis=1)
proc

,0,5,7,8,13,15,16,17,18,19,...,141,142,143,144,145,146,147,148,149,150
0,62.0,0.21875,-9.28125,-9.28125,-6.78125,0.87500,-9.65625,-9.03125,-7.46875,-8.71875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34.0,-9.68750,9.93750,9.53125,9.93750,0.40625,3.71875,9.65625,-2.68750,-9.56250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18.0,-9.84375,-9.84375,-7.21875,-2.03125,-9.93750,-9.96875,-9.87500,-9.81250,-9.78125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,82.0,6.90625,4.75000,-5.90625,-0.40625,-4.03125,3.87500,6.21875,5.65625,6.09375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27.0,-0.03125,-9.09375,-0.40625,7.50000,-7.21875,-9.43750,0.12500,-9.15625,3.65625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54900,13.0,NaN,-6.53125,-2.34375,1.43750,-4.37500,-9.90625,-4.06250,-3.93750,-9.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54901,8.0,NaN,8.93750,9.78125,9.53125,9.50000,9.93750,9.87500,9.96875,9.96875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54902,27.0,NaN,-1.59375,4.53125,-9.21875,-3.09375,-1.96875,1.12500,-5.93750,-4.34375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54903,8.0,NaN,-7.40625,6.93750,-8.00000,-5.40625,-0.03125,5.15625,-3.62500,4.15625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
proc.notnull().sum().min()

166

In [12]:
proc.max(axis=1)

0        62.00000
1        34.00000
2        18.00000
3        82.00000
4        27.00000
           ...   
54900    13.00000
54901     9.96875
54902    27.00000
54903     8.00000
54904    12.00000
Length: 54905, dtype: float64

In [13]:
proc.values

array([[62.     ,  0.21875, -9.28125, ...,      nan,      nan,      nan],
       [34.     , -9.6875 ,  9.9375 , ...,      nan,      nan,      nan],
       [18.     , -9.84375, -9.84375, ...,      nan,      nan,      nan],
       ...,
       [27.     ,      nan, -1.59375, ...,      nan,      nan,      nan],
       [ 8.     ,      nan, -7.40625, ...,      nan,      nan,      nan],
       [12.     ,      nan,  4.25   , ...,      nan,      nan,      nan]])

In [14]:
#how to change it to 0-1 rating ; https://stackoverflow.com/questions/19766757/replacing-numpy-elements-if-condition-is-met
proc.values

array([[62.     ,  0.21875, -9.28125, ...,      nan,      nan,      nan],
       [34.     , -9.6875 ,  9.9375 , ...,      nan,      nan,      nan],
       [18.     , -9.84375, -9.84375, ...,      nan,      nan,      nan],
       ...,
       [27.     ,      nan, -1.59375, ...,      nan,      nan,      nan],
       [ 8.     ,      nan, -7.40625, ...,      nan,      nan,      nan],
       [12.     ,      nan,  4.25   , ...,      nan,      nan,      nan]])

In [15]:
zero_one_mat=pd.DataFrame((proc.values > 0).astype(int), index=proc.index, columns=proc.columns)
zero_one_mat

,0,5,7,8,13,15,16,17,18,19,...,141,142,143,144,145,146,147,148,149,150
0,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54900,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54901,1,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
54902,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
54903,1,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [26]:
class Encoder(nn.Module):
    """
    Function to define SVAE encoder module
    """
    def __init__(self, hyper_params):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(
            hyper_params['rnn_size'], hyper_params['hidden_size']
        )
        nn.init.xavier_normal(self.linear1.weight)
        self.activation = nn.Tanh()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        return x

In [27]:
class Decoder(nn.Module):
    """
    Function to define SVAE decoder module
    """
    def __init__(self, hyper_params):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(hyper_params['latent_size'], hyper_params['hidden_size'])
        self.linear2 = nn.Linear(hyper_params['hidden_size'], hyper_params['total_items'])
        nn.init.xavier_normal(self.linear1.weight)
        nn.init.xavier_normal(self.linear2.weight)
        self.activation = nn.Tanh()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        return x

In [28]:
class Model(nn.Module):
    """
    Function to build the SVAE model
    """
    def __init__(self, hyper_params):
        super(Model, self).__init__()
        self.hyper_params = hyper_params

        self.encoder = Encoder(hyper_params)
        self.decoder = Decoder(hyper_params)

        # Since we don't need padding, our vocabulary size will be "hyper_params['total_items']"
        # and not "hyper_params['total_items'] + 1"
        self.item_embed = nn.Embedding(hyper_params['total_items'], hyper_params['item_embed_size'])

        self.gru = nn.GRU(
            hyper_params['item_embed_size'], hyper_params['rnn_size'],
            batch_first=True, num_layers=1
        )

        self.linear1 = nn.Linear(hyper_params['hidden_size'], 2 * hyper_params['latent_size'])
        nn.init.xavier_normal(self.linear1.weight)

        self.tanh = nn.Tanh()

    def sample_latent(self, h_enc):
        """
        Return the latent normal sample z ~ N(mu, sigma^2)
        """
        temp_out = self.linear1(h_enc)

        mu = temp_out[:, :self.hyper_params['latent_size']]
        log_sigma = temp_out[:, self.hyper_params['latent_size']:]

        sigma = torch.exp(log_sigma)
        std_z = torch.from_numpy(np.random.normal(0, 1, size=sigma.size())).float()

        self.z_mean = mu
        self.z_log_sigma = log_sigma

        return mu + sigma * Variable(std_z, requires_grad=False)  # Reparameterization trick

    def forward(self, x):
        """
        Function to do a forward pass
        :param x: the input
        """
        in_shape = x.shape  # [bsz x seq_len] = [1 x seq_len]
        x = x.view(-1)  # [seq_len]

        x = self.item_embed(x)  # [seq_len x embed_size]
        x = x.view(in_shape[0], in_shape[1], -1)  # [1 x seq_len x embed_size]

        rnn_out, _ = self.gru(x)  # [1 x seq_len x rnn_size]
        rnn_out = rnn_out.view(in_shape[0] * in_shape[1], -1)  # [seq_len x rnn_size]

        enc_out = self.encoder(rnn_out)  # [seq_len x hidden_size]
        sampled_z = self.sample_latent(enc_out)  # [seq_len x latent_size]

        dec_out = self.decoder(sampled_z)  # [seq_len x total_items]
        dec_out = dec_out.view(in_shape[0], in_shape[1], -1)  # [1 x seq_len x total_items]

        return dec_out, self.z_mean, self.z_log_sigma

In [29]:
class VAELoss(torch.nn.Module):
    """
    Function to calculate the VAE loss
    """
    def __init__(self, hyper_params):
        super(VAELoss, self).__init__()
        self.hyper_params = hyper_params

    def forward(self, decoder_output, mu_q, logvar_q, y_true_s, anneal):
        # Calculate KL Divergence loss
        kld = torch.mean(torch.sum(0.5 * (-logvar_q + torch.exp(logvar_q) + mu_q ** 2 - 1), -1))

        # Calculate Likelihood
        dec_shape = decoder_output.shape  # [batch_size x seq_len x total_items] = [1 x seq_len x total_items]

        decoder_output = F.log_softmax(decoder_output, -1)
        num_ones = float(torch.sum(y_true_s[0, 0]))

        likelihood = torch.sum(
            -1.0 * y_true_s.view(dec_shape[0] * dec_shape[1], -1) * \
            decoder_output.view(dec_shape[0] * dec_shape[1], -1)
        ) / (float(self.hyper_params['batch_size']) * num_ones)

        # Calculate the final VAE loss
        final = (anneal * kld) + likelihood

        return final

In [31]:
!pip install comet_ml
from comet_ml import Experiment

# Import packages
import os
import torch
import datetime as dt
import time
import matplotlib.pyplot as plt

Unable to create process using 'C:\Users\Zhao Yunjie\Documents\Anaconda\python.exe "C:\Users\Zhao Yunjie\Documents\Anaconda\Scripts\pip-script.py" install comet_ml'


ModuleNotFoundError: No module named 'comet_ml'

In [32]:
experiment = Experiment(api_key="GaicgDHvizDRCbpq2wVV8NHnX", project_name="autoencoders-movielens1M")

# Dictionary to store model hyper-parameters
hyper_params = {
    'project_name': 'svae_ml1m',  # Project name
    'model_file_name': '',  # Model file name
    'log_file': '',  # Log file name
    'history_split_test': [0.8, 0.2],  # Part of test history to train on : Part of test history to test
    'learning_rate': 0.01,  # Choice of learning rate (required only if optimizer is AdaGrad)
    'optimizer': 'adam',  # Choice of optimizer defaulted to Adam
    'weight_decay': float(5e-3),  # Choice of weight decay defaulted to 0.005
    'epochs': 50,  # Number of epochs
    'batch_size': 1,  # Needs to be 1, because we don't pack multiple sequences in the same batch
    'item_embed_size': 256,  # Item embedding layer of size 256
    'rnn_size': 200,  # Recurrent layer realized as a GRU with 200 cells
    'hidden_size': 150,  # Encoding layer of size 150
    'latent_size': 64,  # Number of latent factors set to 64
    'loss_type': 'next_k',  # [predict_next, same, prefix, postfix, exp_decay, next_k]
    'next_k': 4,  # Size for the number of items forward in time to predict on
    'number_users_to_keep': 1000000000,  # Number of held-out users for evaluation purpose
    'batch_log_interval': 1000,  # Log metrics after this number of batches
    'train_cp_users': 200,
    'exploding_clip': 0.25,  # Exploding gradient clipping
}

# Log model hyper-parameters
experiment.log_parameters(hyper_params)

NameError: name 'Experiment' is not defined

In [ ]:
file_name = '_optimizer_' + str(hyper_params['optimizer'])
if hyper_params['optimizer'] == 'adagrad':
    file_name += '_lr_' + str(hyper_params['learning_rate'])
file_name += '_weight_decay_' + str(hyper_params['weight_decay'])
file_name += '_loss_type_' + str(hyper_params['loss_type'])
file_name += '_item_embed_size_' + str(hyper_params['item_embed_size'])
file_name += '_rnn_size_' + str(hyper_params['rnn_size'])
file_name += '_latent_size_' + str(hyper_params['latent_size'])

# Path to store the log file and the model file
log_file_root = "saved_logs/"
model_file_root = "saved_models/"
if not os.path.isdir(log_file_root):
    os.mkdir(log_file_root)
if not os.path.isdir(model_file_root):
    os.mkdir(model_file_root)
hyper_params['log_file'] = log_file_root + hyper_params['project_name'] + '_log' + file_name + '.txt'
hyper_params['model_file_name'] = model_file_root + hyper_params['project_name'] + '_model' + file_name + '.pt'


In [ ]:
train_reader, val_reader, test_reader, total_items = load_data(hyper_params)
hyper_params['total_items'] = total_items
hyper_params['testing_batch_limit'] = test_reader.num_b

file_write(hyper_params['log_file'], "\n\nSimulation run on: " + str(dt.datetime.now()) + "\n\n")
file_write(hyper_params['log_file'], "Data reading complete!")
file_write(hyper_params['log_file'], "Number of train batches: {:4d}".format(train_reader.num_b))
file_write(hyper_params['log_file'], "Number of validation batches: {:4d}".format(val_reader.num_b))
file_write(hyper_params['log_file'], "Number of test batches: {:4d}".format(test_reader.num_b))
file_write(hyper_params['log_file'], "Total Items: " + str(total_items) + "\n")

In [ ]:
model = Model(hyper_params)
# Loss function is the VAE loss
criterion = VAELoss(hyper_params)

# Different options for the optimizer
if hyper_params['optimizer'] == 'adagrad':
    # AdaGrad
    optimizer = torch.optim.Adagrad(
        model.parameters(), weight_decay=hyper_params['weight_decay'], lr=hyper_params['learning_rate']
    )
elif hyper_params['optimizer'] == 'adadelta':
    # AdaDelta
    optimizer = torch.optim.Adadelta(
        model.parameters(), weight_decay=hyper_params['weight_decay']
    )
elif hyper_params['optimizer'] == 'adam':
    # Adam
    optimizer = torch.optim.Adam(
        model.parameters(), weight_decay=hyper_params['weight_decay']
    )
elif hyper_params['optimizer'] == 'rmsprop':
    # RMSProp
    optimizer = torch.optim.RMSprop(
        model.parameters(), weight_decay=hyper_params['weight_decay']
    )
file_write(hyper_params['log_file'], str(model))
file_write(hyper_params['log_file'], "\nModel Built!\nStarting Training...\n")

best_val_ndcg = None

In [ ]:
try:
    for epoch in range(1, hyper_params['epochs'] + 1):
        # Keep track the time for every epoch
        epoch_start_time = time.time()

        # Perform training
        train(model, criterion, train_reader, optimizer, epoch, hyper_params, experiment)

        # Calculate the metrics on the train set
        metrics, _ = evaluate(model, criterion, train_reader, hyper_params, True, experiment)

        string = ""
        for m in metrics:
            string += " | " + m + ' = ' + str(metrics[m])
        string += ' (TRAIN)'

        # Calculate the metrics on the validation set
        metrics, _ = evaluate(model, criterion, val_reader, hyper_params, False, experiment)

        string2 = ""
        for m in metrics:
            string2 += " | " + m + ' = ' + str(metrics[m])
        string2 += ' (VAL)'

        ss = '-' * 89
        ss += '\n| end of epoch {:3d} | time: {:5.2f}s'.format(epoch, (time.time() - epoch_start_time))
        ss += string
        ss += '\n'
        ss += '-' * 89
        ss += '\n| end of epoch {:3d} | time: {:5.2f}s'.format(epoch, (time.time() - epoch_start_time))
        ss += string2
        ss += '\n'
        ss += '-' * 89
        file_write(hyper_params['log_file'], ss)

        # Log the best NDCG@100 metric on the validation set
        if not best_val_ndcg or metrics['NDCG@100'] >= best_val_ndcg:
            with open(hyper_params['model_file_name'], 'wb') as f:
                torch.save(model, f)
            best_val_ndcg = metrics['NDCG@100']

except KeyboardInterrupt:
    print('Exiting from training early')

In [ ]:
f = open(model.hyper_params['log_file'])
lines = f.readlines()
lines.reverse()

train = []
test = []

for line in lines:
    if line[:10] == 'Simulation' and len(train) > 1:
        break
    elif line[:10] == 'Simulation' and len(train) <= 1:
        train, test = [], []

    if line[2:5] == 'end' and line[-5:-2] == 'VAL':
        test.append(line.strip().split("|"))
    elif line[2:5] == 'end' and line[-7:-2] == 'TRAIN':
        train.append(line.strip().split("|"))


In [ ]:
train.reverse()
test.reverse()

train_ndcg = []
test_ndcg = []
test_loss, train_loss = [], []

for i in train:
    for metric in i:
        if metric.split("=")[0] == " NDCG@100 ":
            train_ndcg.append(float(metric.split('=')[1].split(' ')[1]))
        if metric.split("=")[0] == " loss ":
            train_loss.append(float(metric.split("=")[1].split(' ')[1]))

total, avg_runtime = 0.0, 0.0

for i in test:
    avg_runtime += float(i[2].split(" ")[2][:-1])
    total += 1.0

    for metric in i:
        if metric.split("=")[0] == " NDCG@100 ":
            test_ndcg.append(float(metric.split('=')[1].split(' ')[1]))
        if metric.split("=")[0] == " loss ":
            test_loss.append(float(metric.split("=")[1].split(' ')[1]))


In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 5))
ax1.set_title(hyper_params["project_name"], fontweight="bold", size=20)
ax1.plot(test_ndcg, 'b-')
ax1.set_xlabel('Epochs', fontsize=20.0)
ax1.set_ylabel('NDCG@100', color='b', fontsize=20.0)
ax1.tick_params('y', colors='b')

ax2 = ax1.twinx()
ax2.plot(test_loss, 'r--')
ax2.set_ylabel('Loss', color='r')
ax2.tick_params('y', colors='r')

fig.tight_layout()
if not os.path.isdir("saved_plots/"):
    os.mkdir("saved_plots/")
fig.savefig("saved_plots/learning_curve_" + hyper_params["project_name"] + ".png")
plt.show()

# Log figure
experiment.log_figure(figure=plt)


In [ ]:
with open(hyper_params['model_file_name'], 'rb') as f:
    model = torch.load(f)

# Calculate the metrics on the test set
metrics, len_to_ndcg_at_100_map = evaluate(model, criterion, test_reader, hyper_params, False, experiment)

# Plot sequence length vs NDCG@100 graph
plot_len_vs_ndcg(len_to_ndcg_at_100_map, experiment)

string = ""
for m in metrics:
    string += " | " + m + ' = ' + str(metrics[m])

ss = '=' * 89
ss += '\n| End of training'
ss += string + " (TEST)"
ss += '\n'
ss += '=' * 89
file_write(hyper_params['log_file'], ss)
print("average runtime per epoch =", round(avg_runtime / float(total), 4), "s")

In [16]:
zero_one_mat.sum(axis=1)

0        44
1        25
2         4
3        75
4        11
         ..
54900     6
54901     9
54902     9
54903     4
54904    13
Length: 54905, dtype: int64

In [17]:
np.array(np.where(zero_one_mat.sum(axis=1) <5)).shape

(1, 7260)

In [18]:
zero_one_mat=zero_one_mat.iloc[np.where(zero_one_mat.sum(axis=1) >4)]
zero_one_mat # matrix only having users rated >4 jokes

,0,5,7,8,13,15,16,17,18,19,...,141,142,143,144,145,146,147,148,149,150
0,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54899,1,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
54900,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54901,1,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
54902,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
zero_one_mat.to_pickle('zero_one_mat_filtered.pkl')

In [20]:
good_users_list=zero_one_mat.index

In [21]:
good_users_list

Int64Index([    0,     1,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            54893, 54894, 54895, 54896, 54897, 54899, 54900, 54901, 54902,
            54904],
           dtype='int64', length=47645)

In [22]:
proc_filtered=proc.iloc[good_users_list]
proc_filtered #numeric ratings only contining good users 

,0,5,7,8,13,15,16,17,18,19,...,141,142,143,144,145,146,147,148,149,150
0,62.0,0.21875,-9.28125,-9.28125,-6.78125,0.87500,-9.65625,-9.03125,-7.46875,-8.71875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34.0,-9.68750,9.93750,9.53125,9.93750,0.40625,3.71875,9.65625,-2.68750,-9.56250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,82.0,6.90625,4.75000,-5.90625,-0.40625,-4.03125,3.87500,6.21875,5.65625,6.09375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27.0,-0.03125,-9.09375,-0.40625,7.50000,-7.21875,-9.43750,0.12500,-9.15625,3.65625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,46.0,-2.90625,-2.34375,-0.50000,-0.96875,2.25000,0.40625,0.87500,0.40625,1.43750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54899,14.0,NaN,3.62500,-8.40625,-7.40625,5.28125,-8.93750,3.12500,-7.09375,-9.40625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54900,13.0,NaN,-6.53125,-2.34375,1.43750,-4.37500,-9.90625,-4.06250,-3.93750,-9.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54901,8.0,NaN,8.93750,9.78125,9.53125,9.50000,9.93750,9.87500,9.96875,9.96875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54902,27.0,NaN,-1.59375,4.53125,-9.21875,-3.09375,-1.96875,1.12500,-5.93750,-4.34375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
proc_filtered.max(axis=1).min() #negative users are gone

3.1875

In [24]:
proc_filtered.to_pickle('filtered_rating_mat.pkl') #should I reset the user idx?(think it's better to reset)

## Leave one out test split

In [30]:
#cut off by certain threshold (rating >5)
arr=np.array(jester.iloc[0])
thres=5
np.array(np.where(arr > thres))+1 #indices of elements with positive ratings

array([[ 27,  29,  31,  32,  35,  36,  50,  53,  54,  65,  66,  69,  72,
         76,  83,  87,  89,  91,  93, 108, 109, 119, 120, 121, 123, 127]])

In [93]:
train_matrix=jester.copy()

In [96]:
row=2
arr=jester.iloc[row].fillna(-1000)
top_k_idx=np.argsort(arr)
top_k_idx

1        0
2       94
3       95
4       96
5       97
      ... 
146     12
147     38
148     37
149    101
150    104
Name: 2, Length: 150, dtype: int64

In [97]:
np.array(top_k_idx[::-1][:10])

array([104, 101,  37,  38,  12,  19,   7,  18,  33,  32])

In [100]:
train_matrix.iloc[row,104]

1.25

In [103]:
jester.max(axis=1).min() #wtf such a negative user

-10.0

In [193]:
true_cols=proc_filtered.columns[1:]
true_cols

Index(['5', '7', '8', '13', '15', '16', '17', '18', '19', '20',
       ...
       '141', '142', '143', '144', '145', '146', '147', '148', '149', '150'],
      dtype='object', length=140)

In [179]:
proc_filtered=proc_filtered.reset_index()

In [194]:
proc_filtered.drop(columns=['index'],inplace=True)

In [196]:
proc_filtered.columns=range(0,140)

In [198]:
train_matrix=proc_filtered.copy()

In [199]:
train_matrix

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,0.21875,-9.28125,-9.28125,-6.78125,0.87500,-9.65625,-9.03125,-7.46875,-8.71875,-9.15625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-9.68750,9.93750,9.53125,9.93750,0.40625,3.71875,9.65625,-2.68750,-9.56250,-9.12500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.90625,4.75000,-5.90625,-0.40625,-4.03125,3.87500,6.21875,5.65625,6.09375,5.40625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.03125,-9.09375,-0.40625,7.50000,-7.21875,-9.43750,0.12500,-9.15625,3.65625,-9.43750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-2.90625,-2.34375,-0.50000,-0.96875,2.25000,0.40625,0.87500,0.40625,1.43750,-0.03125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44796,NaN,3.62500,-8.40625,-7.40625,5.28125,-8.93750,3.12500,-7.09375,-9.40625,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44797,NaN,-6.53125,-2.34375,1.43750,-4.37500,-9.90625,-4.06250,-3.93750,-9.00000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44798,NaN,8.93750,9.78125,9.53125,9.50000,9.93750,9.87500,9.96875,9.96875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44799,NaN,-1.59375,4.53125,-9.21875,-3.09375,-1.96875,1.12500,-5.93750,-4.34375,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [200]:
#ndcg: just row-to-row comparision btw gt and pred with rows full of 0,1s
#code seems okay(need some sanity checks), but we should filter out some inappropriate users/jokes
#do some sanity checks
import random
k=5
test_jokes=[]
for row in proc_filtered.index:
  #arr=np.array(jester.iloc[row].dropna())
  arr=proc_filtered.iloc[row].fillna(-1000)
  top_k_idx=np.array(np.argsort(arr)[::-1][:k]) #top k indices
  #print(top_k_idx)
  chosen_test=random.choice(top_k_idx)
  test_jokes.append(chosen_test)
  #print(train_matrix.iloc[row,chosen_test])
  train_matrix.iloc[row,chosen_test]=np.nan

In [201]:
row

44800

Think it works well!!

In [202]:
train_matrix

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,0.21875,-9.28125,-9.28125,-6.78125,0.87500,-9.65625,-9.03125,-7.46875,-8.71875,-9.15625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-9.68750,9.93750,9.53125,9.93750,0.40625,3.71875,9.65625,-2.68750,-9.56250,-9.12500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.90625,4.75000,-5.90625,-0.40625,-4.03125,3.87500,6.21875,5.65625,6.09375,5.40625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.03125,-9.09375,-0.40625,7.50000,-7.21875,-9.43750,0.12500,-9.15625,3.65625,-9.43750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-2.90625,-2.34375,-0.50000,-0.96875,2.25000,0.40625,0.87500,0.40625,1.43750,-0.03125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44796,NaN,NaN,-8.40625,-7.40625,5.28125,-8.93750,3.12500,-7.09375,-9.40625,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44797,NaN,-6.53125,-2.34375,1.43750,-4.37500,-9.90625,-4.06250,-3.93750,-9.00000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44798,NaN,8.93750,9.78125,9.53125,9.50000,NaN,9.87500,9.96875,9.96875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44799,NaN,-1.59375,NaN,-9.21875,-3.09375,-1.96875,1.12500,-5.93750,-4.34375,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
(train_matrix.isnull().sum(axis=1) - proc_filtered.isnull().sum(axis=1)).sum() #sanity check, should be equal to num of rows

44801

In [216]:
train_matrix.columns=true_cols

In [218]:
train_matrix.to_pickle('masked_for_train.pkl')

In [219]:
proc_filtered.columns=true_cols

In [220]:
proc_filtered.to_pickle('gt_matrix.pkl')

In [ ]:
test_jokes

In [208]:
len(test_jokes)

44801

In [209]:
true_cols

Index(['5', '7', '8', '13', '15', '16', '17', '18', '19', '20',
       ...
       '141', '142', '143', '144', '145', '146', '147', '148', '149', '150'],
      dtype='object', length=140)

In [210]:
proc_filtered.columns

RangeIndex(start=0, stop=140, step=1)

In [214]:
columns_map_dict=dict(zip(proc_filtered.columns, true_cols))

In [223]:
test_jokes_mapped=[columns_map_dict[k] for k in test_jokes]

In [224]:
len(test_jokes_mapped)

44801

In [225]:
proc_filtered['test_joke_idx']=test_jokes_mapped

In [226]:
proc_filtered.to_pickle('gt_matrix.pkl')

In [227]:
proc_filtered

,5,7,8,13,15,16,17,18,19,20,...,142,143,144,145,146,147,148,149,150,test_joke_idx
0,0.21875,-9.28125,-9.28125,-6.78125,0.87500,-9.65625,-9.03125,-7.46875,-8.71875,-9.15625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109
1,-9.68750,9.93750,9.53125,9.93750,0.40625,3.71875,9.65625,-2.68750,-9.56250,-9.12500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
2,6.90625,4.75000,-5.90625,-0.40625,-4.03125,3.87500,6.21875,5.65625,6.09375,5.40625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
3,-0.03125,-9.09375,-0.40625,7.50000,-7.21875,-9.43750,0.12500,-9.15625,3.65625,-9.43750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21
4,-2.90625,-2.34375,-0.50000,-0.96875,2.25000,0.40625,0.87500,0.40625,1.43750,-0.03125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44796,NaN,3.62500,-8.40625,-7.40625,5.28125,-8.93750,3.12500,-7.09375,-9.40625,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
44797,NaN,-6.53125,-2.34375,1.43750,-4.37500,-9.90625,-4.06250,-3.93750,-9.00000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69
44798,NaN,8.93750,9.78125,9.53125,9.50000,9.93750,9.87500,9.96875,9.96875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16
44799,NaN,-1.59375,4.53125,-9.21875,-3.09375,-1.96875,1.12500,-5.93750,-4.34375,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8


In [237]:
test_rating=[]
for i in range(44801):
  idx=str(proc_filtered.loc[i,'test_joke_idx'])
  test_rating.append(proc_filtered.loc[i,idx])

In [240]:
min(test_rating)

0.03125

# Evaluation

In [ ]:
from sklearn.metrics import ndcg_score

In [ ]:
user_id=0
ndcg_score(np.asarray([df_processed.loc[user_id]]), np.asarray([restored.loc[user_id]]))

0.8876414707651327

In [ ]:
arr=np.array([4,5,1,3])
arr.argsort()[::-1].argsort()+1

array([2, 1, 4, 3])

In [ ]:
def get_rank(arr):
  return arr.argsort()[::-1].argsort()+1

In [ ]:
gt_rank=get_rank(df_processed.loc[user_id])
pred_rank=get_rank(restored.loc[user_id])

In [ ]:
np.asarray([gt_rank])

array([[150,  72,  73,  74,  35,  75,  60,  61,  76,  77,  78,  79,  50,
         80,  32,  62,  58,  52,  56,  59,  51,  57,  55,  54,  53,  43,
         10,  81,  11,  82,  13,  14,  84,  48,  16,  17,  85,  86,  87,
         88,  89,  39,  90,  71,  70,  67,  66,  63,  40,   1,  41,  37,
         12,  24,  68,  65,  64,  91,  83,  93,  42,  34, 108, 125,  18,
         22, 126,  46,  21, 127, 128,  15, 129, 130,  92,   5, 132, 133,
        134,  38,  36, 135,   6, 136, 137, 138,  25, 139,   2, 140,  19,
         27,   4, 141, 142, 143, 144, 145, 146, 147, 148,  33,  49,  29,
         31,  47,  30,  26,   3, 149, 124, 123, 122, 121,  95,  96,  97,
         44,   8,   7,  20,  98,   9,  99, 100, 101,  23,  45, 102, 103,
        104, 105, 106,  28,  94, 107, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120, 131,  69]])

In [ ]:
ndcg_score(np.asarray([gt_rank]), np.asarray([pred_rank]), k=10)

0.5190368261143826

In [ ]:
ndcg_cum=0
for user_id in range(len(df_processed)):
  gt_rank=get_rank(df_processed.loc[user_id])
  pred_rank=get_rank(restored.loc[user_id])
  ndcg_cum+=ndcg_score(np.asarray([gt_rank]), np.asarray([pred_rank]), k=10)

In [ ]:
ndcg_total=ndcg_cum/len(df_processed)
ndcg_total #for k=5

0.605376949965302

In [ ]:
ndcg_total=ndcg_cum/len(df_processed)
ndcg_total #for k=10

0.6078611202443114

In [ ]:
hit=0
k=10
for user_id in range(len(df_processed)):
  top_k_pred=list(restored.loc[user_id].sort_values(ascending=False).index[:k])
  top_true=df_processed.loc[user_id].sort_values(ascending=False).index[0]
  if top_true in top_k_pred:
    hit+=1

In [ ]:
hit_ratio=hit/len(df_processed)
hit_ratio #for k=5

0.30601948820690283

In [ ]:
hit_ratio=hit/len(df_processed)
hit_ratio #for k=10

0.5148893543393134

## Naive approach evaluation

In [ ]:
naive_pred=df_processed.mean()
naive_pred

1       0.000000
2       0.000000
3       0.000000
4       0.000000
5       0.288287
         ...    
146     2.824381
147     5.914830
148     9.696221
149     6.799298
150    11.493085
Length: 150, dtype: float64

In [ ]:
#ndcg
ndcg_cum=0
naive_rank=get_rank(naive_pred)
for user_id in range(len(df_processed)):
  gt_rank=get_rank(df_processed.loc[user_id])
  ndcg_cum+=ndcg_score(np.asarray([gt_rank]), np.asarray([naive_rank]), k=10)

In [ ]:
ndcg_total=ndcg_cum/len(df_processed)
ndcg_total #for k=5

0.6135019662129706

In [ ]:
ndcg_total=ndcg_cum/len(df_processed)
ndcg_total #for k=10

0.5629028558834749

In [ ]:
#hit ratio
hit=0
k=10
naive_top_k_pred=list(naive_pred.sort_values(ascending=False).index[:k])
for user_id in range(len(df_processed)):
  top_true=df_processed.loc[user_id].sort_values(ascending=False).index[0]
  if top_true in naive_top_k_pred:
    hit+=1

In [ ]:
hit_ratio=hit/len(df_processed)
hit_ratio #for k=5

0.260304161733904

In [ ]:
hit_ratio=hit/len(df_processed)
hit_ratio #for k=10

0.44050632911392407

## Below here is just for reference
- Codes we can refer for more advanced models:
- https://github.com/microsoft/recommenders
- https://github.com/microsoft/recommenders/blob/main/examples/02_model_collaborative_filtering/lightgcn_deep_dive.ipynb
- https://github.com/microsoft/recommenders/blob/main/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb


In [ ]:
df_svd_preds = pd.DataFrame(svd_user_predicted_link,index= ply_pivot1.index ,columns = ply_pivot1.columns)
df_svd_preds

In [ ]:
#우리가 캡스톤에 사용했던 방식은 결국 leave one out인거암. 
def recommend_songs(df_svd_preds, user_id, ori_link_df, num_recommendations):
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 곡 데이터 정렬 -> 영화 유사도가 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[user_id].sort_values(ascending=False)
    # 원본 곡 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    #user_data = ori_link_df.loc[user_id]
    # 사용자의 곡 유사도가 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = pd.DataFrame(sorted_user_predictions).reset_index()
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
    return recommendations

In [ ]:
predictions = recommend_songs(df_svd_preds, 23431, ply_pivot1, 100)

In [ ]:
predictions

In [ ]:
import json
 
# Ready for data
songs = []
song = {}
for name in ply_pivot_val.id:
  song['id'] = name
  song['songs'] = list(recommend_songs(df_svd_preds, name, ply_pivot1, 100)['song_id'])
  songs.append(song)

 
# Print JSON
print(json.dumps(songs, indent="\t") )

In [ ]:
# Write JSON
with open('./답안/res.json', 'w', encoding="utf-8") as make_file:
    json.dump(songs, make_file, ensure_ascii=False, indent="\t")

In [ ]:
f = open('./답안/res.json', encoding="UTF-8")
raw_data = json.loads(f.read())
raw_data